In [1]:
!pip uninstall -q tensorflow tensorflow-estimator tensorboard tensorflow-probability 
!pip install -q tensorflow==2.1.0 

Proceed (y/n)? y
y
y
y
y
     |████████████████████████████████| 421.8 MB 25 kB/s 
     |████████████████████████████████| 448 kB 57.6 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
     |████████████████████████████████| 3.8 MB 36.0 MB/s 


In [ ]:
import tensorflow as tf 
import tensorflow.keras as keras 
import numpy as np

print(tf.__version__)
print(tf.test.gpu_device_name())

2.1.0
/device:GPU:0


In [ ]:
with open('data.txt', 'r', encoding='utf-8') as f: #한국어 파일을 열기 위해 encoding='utf-8'필요
  data_text = f.read()

In [ ]:
len(data_text)

166939

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=False, char_level=True) 
tokenizer.fit_on_texts(data_text)

In [ ]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters
print(max_id)
print(dataset_size)

1771
166939


In [ ]:
[encoded] = np.array(tokenizer.texts_to_sequences([data_text])) - 1

In [ ]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [ ]:
n_steps = 100
window_length = n_steps + 1 # target <= input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [ ]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [ ]:
dataset = dataset.prefetch(1)

In [ ]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 1771) (32, 100)


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2), #recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2), #recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [ ]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size, #에포크를 더 늘리면 정확도가 오를것으로 보이나 코랩이 자꾸 튕겨서 더 진행하지 못했습니다ㅠ
                    epochs=30)

Train for 4695 steps
Epoch 1/30
4695/4695 [==============================] - 179s 38ms/step - loss: 3.8583 - accuracy: 0.3200
Epoch 2/30
4695/4695 [==============================] - 178s 38ms/step - loss: 2.5043 - accuracy: 0.4722
Epoch 3/30
4695/4695 [==============================] - 176s 37ms/step - loss: 2.1657 - accuracy: 0.5219
Epoch 4/30
4695/4695 [==============================] - 175s 37ms/step - loss: 1.9895 - accuracy: 0.5505
Epoch 5/30
4695/4695 [==============================] - 177s 38ms/step - loss: 1.8788 - accuracy: 0.5694
Epoch 6/30
4695/4695 [==============================] - 176s 37ms/step - loss: 1.8021 - accuracy: 0.5830
Epoch 7/30
4695/4695 [==============================] - 177s 38ms/step - loss: 1.7469 - accuracy: 0.5930
Epoch 8/30
4695/4695 [==============================] - 177s 38ms/step - loss: 1.7040 - accuracy: 0.6012
Epoch 9/30
4695/4695 [==============================] - 178s 38ms/step - loss: 1.6704 - accuracy: 0.6072
Epoch 10/30
4695/4695 [===========

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [ ]:
X_new = preprocess(["야 이 사람"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

'아'

In [ ]:
tf.random.set_seed(42)
# https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/random/categorical 
tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=5).numpy()

array([[0, 0, 1, 1, 1]], dtype=int64)

In [ ]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
tf.random.set_seed(42)

next_char("야 이 사람", temperature=1)

' '

In [ ]:
next_char("야 이 사람", temperature=0.1)

'아'

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
tf.random.set_seed(42)

print(complete_text("내", temperature=0.1))

내 이 기 전에 사뭇 그들먹합니다.
 얼굴이 도로 비뚤어진 친구는 광대로 인식 착오를 일으키


In [ ]:
print(complete_text("내", temperature=0.5))

내 시어서 버얼떡 누운 채, 윤직원 영감은 옹색한 좌판에서 가까스로 뒤를 쳐들고, 코무에 꺼


In [ ]:
print(complete_text("내", temperature=1))

내에 경손이한테 돈을 50전이면 만원씨의 아저 뒤에 10전심이 을 알고서 서리고 삼백 권이 


In [ ]:
print(complete_text("내", temperature=2))

내환타짜고 죠. 그 이 사람에장은매 군렁개다 되니듯을 듬씬 보채도을 듣럽진 이기면박를 천욱이


In [ ]:
print(complete_text("그", temperature=0.2))

그 니다.
 그렇지만, 그건 아까 이 사람이 행에겠다는 게 천하에 더불어 물러오던 것이니까,


In [ ]:
print(complete_text("그", temperature=0.25))

그 좋은 것이지만, 그러나 참 혹은 옹색하고 조심이 되는 게 아닙니다.
 그러나 그렇게 여기


In [ ]:
print(complete_text("그", temperature=0.5))

그런 길러 약짜 좋아야 할 것이지도 하고, 어디 그는 것은 오줌 특약을 해두고는, 새벽이면 


In [ ]:
print(complete_text("그", temperature=0.75))

그램으로 누시오. 아는 날이라, 그리고 까행 여단운 라고 그래나 못 살고 싶은 게 한번 한푼


In [ ]:
print(complete_text("그", temperature=1))

그니다.
 인력거꾼은 혼자코 실복을 할 뿐입니다.
 대잔, 통치를 찌푸리면서 방을 받습니다.


In [ ]:
print(complete_text("그", temperature=2))

그게운돌저넘어 쏜사음 증타미 들 으로습합입죽참!"사하러 생대은 수조가 듣고 히 하던니다. 을


In [ ]:
print(complete_text("나는", temperature=0.25))

나는 그 전보 좀 건너다니 그 게 명색이 마 소작  인력거꾼은 여태 였었읍니다. 그러나 이렇게
